### Packages

In [1088]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import gym
import scipy.signal
import time
from tensorflow.keras import Model
import matplotlib.pyplot as plt
import random
import tensorflow_probability as tfp

### Env Setup

In [1089]:
problem = "Hopper-v3"
env = gym.make(problem)

num_states = env.observation_space.shape[0]
num_actions = env.action_space.shape[0]
upper_bound = env.action_space.high[0]
lower_bound = env.action_space.low[0]

num_states, num_states, upper_bound, lower_bound

EPSILON = 1e-10

### Actor Model

In [1135]:
class Actor(Model):

    def __init__(self, action_dimensions):
        super().__init__()
        self.action_dim = action_dimensions
        self.sample_dist = tfp.distributions.MultivariateNormalDiag(loc=tf.zeros(num_actions),
                                                                    scale_diag=tf.ones(num_actions))
        self.dense1_layer = layers.Dense(256, activation="relu")
        self.dense2_layer = layers.Dense(256, activation="relu")
        self.mean_layer = layers.Dense(self.action_dim)
        self.stdev_layer = layers.Dense(self.action_dim)

    def call(self, state, eval_mode=False):

        a1 = self.dense1_layer(state)
        a2 = self.dense2_layer(a1)
        mu = self.mean_layer(a2)

        log_sigma = self.stdev_layer(a2)
        sigma = tf.exp(log_sigma)

        dist = tfp.distributions.MultivariateNormalDiag(loc=mu, scale_diag=sigma)

        if eval_mode:
            action_ = mu
        else:
            action_ = tf.math.add(mu, tf.math.multiply(sigma, tf.expand_dims(self.sample_dist.sample(), 0)))
 
        action = tf.tanh(action_)

        log_pi_ = dist.log_prob(action_)     
        log_pi = log_pi_ - tf.reduce_sum(tf.math.log(tf.clip_by_value(1 - action**2, EPSILON, 1.0)), axis=1)
        
        print(sigma)
        s_e = tf.expand_dims(self.sample_dist.sample(), 0)
        print("Sample epsilon: ", s_e)
        print(tf.math.multiply(sigma, s_e))
        return action*upper_bound, log_pi


#### Testing

In [1136]:
actor_test = Actor(num_actions)

In [1137]:
obs = env.reset()
obs
tf_obs = tf.expand_dims(obs, 0)
tf_obs
a_test, log_a_test = actor_test(tf_obs)

tf.Tensor([[1.1533982  0.9330433  0.95782125]], shape=(1, 3), dtype=float32)
Sample epsilon:  tf.Tensor([[ 0.19212519 -0.32455724  0.6135523 ]], shape=(1, 3), dtype=float32)
tf.Tensor([[ 0.22159684 -0.30282596  0.5876734 ]], shape=(1, 3), dtype=float32)


In [1138]:
obs_new, _, _, _ = env.step(a_test[0])
tf_obs_new = tf.expand_dims(obs_new, 0)
a_test, log_a_test = actor_test(tf.convert_to_tensor([tf_obs, tf_obs_new]))

tf.Tensor(
[[[1.1533982  0.9330433  0.95782125]]

 [[1.0369197  0.9678792  0.916226  ]]], shape=(2, 1, 3), dtype=float32)
Sample epsilon:  tf.Tensor([[ 0.00502631  0.7337843  -0.04475469]], shape=(1, 3), dtype=float32)
tf.Tensor(
[[[ 0.00579733  0.68465257 -0.04286699]]

 [[ 0.00521188  0.71021456 -0.04100541]]], shape=(2, 1, 3), dtype=float32)


In [1094]:
actor_test.summary()

Model: "actor_158"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1126 (Dense)          multiple                  3072      
                                                                 
 dense_1127 (Dense)          multiple                  65792     
                                                                 
 dense_1128 (Dense)          multiple                  771       
                                                                 
 dense_1129 (Dense)          multiple                  771       
                                                                 
Total params: 70,406
Trainable params: 70,406
Non-trainable params: 0
_________________________________________________________________


### Critic Model

In [1095]:
class Critic_Wrapper():
    def __init__(self, state_dim):
        self.s_dim=state_dim
        
    def get_critic(self):
        # State as input
        state_input = layers.Input(shape=(self.s_dim))
        state_out = layers.Dense(256, activation="relu")(state_input)
        # state_out = layers.Dense(32, activation="relu")(state_out)

        out = layers.Dense(256, activation="relu")(state_out)
        outputs = layers.Dense(1, dtype='float64')(out)

        # Outputs single value for give state-action
        model = tf.keras.Model([state_input], outputs)

        return model


#### Testing

In [1096]:
critic_gen = Critic_Wrapper(num_states)
critic_test = critic_gen.get_critic()

In [1097]:
obs = env.reset()
obs

array([ 1.25271482e+00,  2.79971789e-03, -1.63619178e-03, -1.33913395e-03,
        1.33965093e-03, -3.43120515e-03,  6.74936643e-04,  4.76680213e-03,
       -4.15028315e-03,  6.28646674e-04, -4.57904844e-03])

In [1098]:
tf_obs = tf.expand_dims(obs, 0)
a_test, log_a_test = actor_test(tf_obs)
tf_obs

<tf.Tensor: shape=(1, 11), dtype=float64, numpy=
array([[ 1.25271482e+00,  2.79971789e-03, -1.63619178e-03,
        -1.33913395e-03,  1.33965093e-03, -3.43120515e-03,
         6.74936643e-04,  4.76680213e-03, -4.15028315e-03,
         6.28646674e-04, -4.57904844e-03]])>

In [1099]:
v_test = tf.squeeze(critic_test([tf_obs]))
v_test

<tf.Tensor: shape=(), dtype=float64, numpy=0.036233229949890575>

In [1100]:
critic_test.summary()

Model: "model_150"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_193 (InputLayer)      [(None, 11)]              0         
                                                                 
 dense_1130 (Dense)          (None, 256)               3072      
                                                                 
 dense_1131 (Dense)          (None, 256)               65792     
                                                                 
 dense_1132 (Dense)          (None, 1)                 257       
                                                                 
Total params: 69,121
Trainable params: 69,121
Non-trainable params: 0
_________________________________________________________________


### Replay Buffer

In [1101]:
class Buffer:

    def __init__(self, observation_dimensions, action_dimensions, size, minibatch_size=256, gamma=0.99, lam=0.95):

        self.observation_buffer = np.zeros(
            (size, observation_dimensions), dtype=np.float32
        )
        self.action_buffer = np.zeros((size, action_dimensions), dtype=np.float32)
        self.reward_buffer = np.zeros(size, dtype=np.float32)
        self.logprobability_buffer = np.zeros(size, dtype=np.float32)
        
        self.gamma, self.lam = gamma, lam
        self.batch_size = minibatch_size
        
        self.buffer_cap = size
        self.pointer = 0
        self.trajectory_start_indices = []
        self.trajectory_start_indices.append(0)

    def store(self, observation, action, reward, logprobability, done):

        self.observation_buffer[self.pointer] = observation
        self.action_buffer[self.pointer] = action
        self.reward_buffer[self.pointer] = reward
        self.logprobability_buffer[self.pointer] = logprobability
        self.pointer += 1
        if done and not self.pointer > self.buffer_cap-1:
            self.trajectory_start_indices.append(self.pointer)


    def get(self):
        # Get all data of the buffer
        if self.trajectory_start_indices[-1] == self.buffer_cap-1:
            rindex = np.random.choice(range(len(self.trajectory_start_indices)-1), self.batch_size)
        else:
            rindex = np.random.choice(range(len(self.trajectory_start_indices)), self.batch_size)
        
        isolated_obs=[]
        isolated_a=[]
        isolated_r=[]
        isolated_log_a=[]
        for ri in rindex:
            
            if  ri == len(self.trajectory_start_indices)-1:
                isolated_obs.append(self.observation_buffer[self.trajectory_start_indices[ri]:
                                                       self.buffer_cap])
                isolated_a.append(self.action_buffer[self.trajectory_start_indices[ri]:
                                                       self.buffer_cap])
                isolated_r.append(self.reward_buffer[self.trajectory_start_indices[ri]:
                                                       self.buffer_cap])
                isolated_log_a.append(self.logprobability_buffer[self.trajectory_start_indices[ri]:
                                                       self.buffer_cap])
                
            else:
                isolated_obs.append(self.observation_buffer[self.trajectory_start_indices[ri]:
                                                       self.trajectory_start_indices[ri+1]])
                isolated_a.append(self.action_buffer[self.trajectory_start_indices[ri]:
                                                       self.trajectory_start_indices[ri+1]])
                isolated_r.append(self.reward_buffer[self.trajectory_start_indices[ri]:
                                                       self.trajectory_start_indices[ri+1]])
                isolated_log_a.append(self.logprobability_buffer[self.trajectory_start_indices[ri]:
                                                       self.trajectory_start_indices[ri+1]])

        return (
            isolated_obs,
            isolated_a,
            isolated_r,
            isolated_log_a,
        )
    
    def batch_sample(self, critic_handle):
        s_b, a_b, r_b, l_b = self.get()
        ss_b = []
        as_b = []
        rs_b = []
        ls_b = []
        adv_b = []
        ret_b = []
        sample_idxs = [np.random.choice(range(len(a)-1)) for a in s_b]
        
        for i in range(self.batch_size):
            ss_b.append(s_b[i][sample_idxs[i]])
            as_b.append(a_b[i][sample_idxs[i]])
            rs_b.append(r_b[i][sample_idxs[i]])
            ls_b.append(l_b[i][sample_idxs[i]])
            adv_b.append(self.adv_t(r_b[i][sample_idxs[i]:-1],
                                      critic_handle,
                                      s_b[i][sample_idxs[i]:-1],
                                      s_b[i][sample_idxs[i]+1:]))
            ret_b.append(self.ret_t(r_b[i][sample_idxs[i]:]))
        return (
            tf.convert_to_tensor(ss_b),
            tf.convert_to_tensor(as_b),
            tf.convert_to_tensor(adv_b),
            tf.convert_to_tensor(ret_b),
            tf.convert_to_tensor(ls_b),
            tf.convert_to_tensor(rs_b)
        )
        
    def adv_t(self, r_t, vf, s_t, s_t1):
        ite_gamma_lam = [(self.gamma*self.lam)**i for i in range(len(r_t))]
        delta_ts = r_t + self.gamma*tf.squeeze(vf(s_t1)) - tf.squeeze(vf(s_t))

        return np.sum(np.multiply(ite_gamma_lam, delta_ts))
    
    def ret_t(self, r_t):
        ite_gamma = [self.gamma**i for i in range(len(r_t))]
        
        return np.sum(np.multiply(ite_gamma, r_t))
    
    def clear(self):
        self.pointer = 0
        self.trajectory_start_indices = []
        self.trajectory_start_indices.append(0)

#### Testing

In [1102]:
buffer = Buffer(num_states, num_actions, 100, 5)
actor_test = Actor(num_actions)
critic_gen = Critic_Wrapper(num_states)
critic_test = critic_gen.get_critic()

In [1103]:
obs = env.reset()
buffer.clear()
for x in range(100):
    tf_obs = tf.expand_dims(obs, 0)
    a, log_a = actor_test(tf_obs)
    a = a[0]
    obs_new, r, d, _ = env.step(a)
    
    buffer.store(obs, a, r, log_a, d)
    if d:
        obs = env.reset()
    else:
        obs = obs_new
    
print(buffer.trajectory_start_indices)

[0, 12, 23, 40, 57, 70, 87]


In [1104]:
s_b, a_b, r_b, l_b = buffer.get()
# np.array(s_b, dtype=object).shape, np.array(a_b, dtype=object).shape, np.array(r_b, dtype=object).shape, np.array(l_b, dtype=object).shape
# np.array(s_b, dtype=object), np.array(a_b, dtype=object), np.array(r_b, dtype=object), np.array(l_b, dtype=object)
# print("trajectory len: ", len(s_b[0][0]))


In [1105]:
buffer.batch_sample(critic_test)

(<tf.Tensor: shape=(5, 11), dtype=float32, numpy=
 array([[ 1.2255454e+00, -2.8130250e-02, -1.5443403e-02, -3.3060860e-02,
         -1.8586377e-02, -1.5734534e-01, -7.4212414e-01, -2.3261580e+00,
         -2.4034238e+00, -7.5640047e-01,  2.0821412e+00],
        [ 1.2080775e+00, -7.9052143e-02, -2.7084652e-02, -1.2352135e-01,
          1.8645383e-01, -8.8629521e-02, -2.2985950e-01, -3.6071446e+00,
         -2.9314358e+00, -2.6402762e+00,  2.1169651e+00],
        [ 1.2152959e+00, -1.2636462e-01, -1.9897884e-02, -2.3831084e-01,
          4.9185038e-02, -8.2101423e-01, -8.2815582e-01, -5.2101607e+00,
         -2.7035944e+00, -5.4663806e+00,  2.3161933e+00],
        [ 1.2222453e+00, -1.4662923e-01, -1.2887014e-01, -8.5827507e-02,
          2.5990418e-01, -2.5638029e-01, -5.7340896e-01, -4.1354628e+00,
         -2.9734170e+00, -4.0964475e+00,  7.6045208e+00],
        [ 1.2417740e+00,  1.4921312e-03,  5.0133835e-03, -2.1059953e-02,
          7.2312012e-02, -8.7250061e-02, -3.9589050e-01, -7.9

### PPO

In [1106]:
class PPO:
    
    def __init__(self, env, observation_dimensions, action_dimensions, horizon,
                 minibatch_size=256, gamma=0.99, lam=0.95, diagnostic_length=1, lr=3e-4):
        
        self.env = env
        self.actor = Actor(action_dimensions)
        self.critic_gen = Critic_Wrapper(observation_dimensions)
        self.critic = self.critic_gen.get_critic()
        self.buffer = Buffer(observation_dimensions, action_dimensions, horizon, minibatch_size, gamma, lam)
        
        self.p_opt= tf.keras.optimizers.Adam(learning_rate=lr,
                                                            )
        self.v_opt= tf.keras.optimizers.Adam(learning_rate=lr,
                                                            )
        self.clip_epsilon = 0.2
        
        self.diagnostics_buffer = []
        self.diagno_index = 0
        self.diagno_length = diagnostic_length
        
        self.gamma, self.lam, self.horizon = gamma, lam, horizon
        
    def train(self, iterations, epochs=20):
        
        for i in range(iterations):
            
            obs = self.env.reset()
            
            for t in range(self.horizon):
                
                tf_obs = tf.expand_dims(obs, 0)
                a, log_a = self.actor(tf_obs)
                a=a[0]
            
                obs_new, r, d, _ = self.env.step(a)
                
                self.buffer.store(obs, a, r, log_a, d)
                
                if d:
                    obs = self.env.reset()
                else:
                    obs = obs_new

            for _ in range(epochs):
                (
                    obs_b,
                    a_b,
                    adv_b,
                    ret_b,
                    log_b,
                    r_b,
                ) = self.buffer.batch_sample(self.critic)
                self.update(obs_b, adv_b, log_b, ret_b)
            self.show_diagnostics()    
            self.buffer.clear()
            
    def update(self, obs_b, adv_b, log_b, ret_b):
        with tf.GradientTape() as tape:
            a, log_a = self.actor(obs_b)
            ratio = tf.exp(log_a - log_b)
            c_ratio = tf.clip_by_value(ratio, 1.0-self.clip_epsilon, 1.0+self.clip_epsilon)

            rt_at = tf.minimum(tf.math.multiply(ratio, tf.cast(adv_b, tf.float32)), 
                               tf.math.multiply(c_ratio, tf.cast(adv_b, tf.float32)))

            L_theta_clip = -tf.reduce_mean(rt_at)
        J_theta_clip = tape.gradient(L_theta_clip, self.actor.trainable_variables)
        self.p_opt.apply_gradients(zip(J_theta_clip, self.actor.trainable_variables))
        
        with tf.GradientTape() as tape1:
            v_theta = tf.squeeze(self.critic(obs_b))
            v_mse = tf.reduce_mean((v_theta - ret_b)**2)
        J_phi = tape1.gradient(v_mse, self.critic.trainable_variables)
        self.v_opt.apply_gradients(zip(J_phi, self.critic.trainable_variables))
        self.record_diagnostics(["policy loss: ", np.array(L_theta_clip), "value loss: ", np.array(v_mse)])
        
    def record_diagnostics(self, data):
        if len(self.diagnostics_buffer) == self.diagno_length:
            self.diagnostics_buffer[self.diagno_index] = data

        if len(self.diagnostics_buffer) < self.diagno_length:
            self.diagnostics_buffer.append(data)
        self.diagno_index = (self.diagno_index+1)%self.diagno_length
    def show_diagnostics(self):
        for i in range(len(self.diagnostics_buffer)):
            print(self.diagnostics_buffer[(self.diagno_index+i)%len(self.diagnostics_buffer)])
            
    def save_weights(self, a_path, c_path):
        raise NotImplementedError

    def load_weights(self, a_path, c_path):
        raise NotImplementedError

#### Testing

In [1107]:
ppo1 = PPO(env, num_states, num_actions, 1000)
ppo1.train(1000, 100)

['policy loss: ', array(-0.95306444, dtype=float32), 'value loss: ', array(20.21711021)]
['policy loss: ', array(-0.62134653, dtype=float32), 'value loss: ', array(56.98488945)]
['policy loss: ', array(-0.6764257, dtype=float32), 'value loss: ', array(34.49099413)]
['policy loss: ', array(-0.46545815, dtype=float32), 'value loss: ', array(15.88138123)]


KeyboardInterrupt: 